In [3]:
from credentials import db
import firebase_admin
import numpy as np
from firebase_admin import firestore
from datetime import datetime

In [4]:
def userDictToList(dict):
    return list(filter(lambda k: k not in ["__user", "__lastFetched", "__len"], dict.keys()))

In [5]:
def convertUserDict(user_dict):
    if user_dict == None:
        return None
    
    return {
        'user': user_dict['__user'],
        'len': user_dict['__len'],
        'lastFetched': user_dict['__lastFetched'],
        'subreddits': userDictToList(user_dict)
    }

def getUsersFromRef(users_ref):
    users = []
    
    for doc in users_ref.stream():
        doc_dict = doc.to_dict()
        users.append(convertUserDict(doc_dict))
        
    return users

In [6]:
def getUsersInTheseSubreddits(subreddits, limit=10):
    users_ref = db.collection('reddit').document('db').collection('users')
    
    for sub in subreddits:
        users_ref = users_ref.where(sub, "==", True)
        
    return getUsersFromRef(users_ref.limit(limit))

In [7]:
def getAverageNumberOfSubredditsPerUser(limit=10):
    users_ref = db.collection('reddit').document('db').collection('users')
    query = users_ref.limit(limit)

    results = query.get()

    subreddits_nummber_arr = []

    for result in results:
        user = userDictToList(result.to_dict())
        subreddits_nummber_arr.append(len(user))
        
    return  np.mean(subreddits_nummber_arr)

In [8]:
def getSubredditsContainingTheseSubreddits(subreddits, limit=10, exclude=True, intersection=True):
    ref = db.collection('reddit').document('db').collection('subreddits')
    results = []
    visited_ids = []
    
    for sub in subreddits:
        query = ref.order_by(sub, direction=firestore.Query.DESCENDING).limit(limit)
        results_g = query.get()
        for r in results_g:
            id = r.id
            neighbours =  r.to_dict()
            extracted_subs =  list(neighbours.keys())
            
            if (exclude and id in subreddits) or (id in visited_ids):
                continue
            
            if intersection:
                for sub2 in subreddits:
                    if sub2 not in extracted_subs:
                        continue
                
            
            results.append({
                'subreddit': id,
                'neighbours': neighbours
            })
            visited_ids.append(id)
            
    return results

# sorted_list = sorted(sub_dict['neighbours'].items(), key=lambda item: item[1], reverse=True)

In [12]:
def extractSubreddit(subreddit):
    ref = db.collection('reddit').document('db').collection('subreddits').document(subreddit)
    doc = ref.get()
    doc_dict = doc.to_dict()
    return sorted(doc_dict.items(), key=lambda item: item[1], reverse=True)

In [25]:
def getUsersBetweenTimeRange(start_time, end_time=datetime.now(), limit=10):
    users_ref = db.collection('reddit').document('db').collection('users')
    query = users_ref.where('__lastFetched', '>=', start_time).where('__lastFetched', '<=', end_time).limit(limit)
    return getUsersFromRef(query)

# getUsersBetweenTimeRange(datetime(2022, 10, 10, 12, 0, 0), datetime(2022, 10, 10, 14, 0, 0))

In [26]:
def getLastFetchedUsers(limit=10):
    users_ref = db.collection('reddit').document('db').collection('users')
    query = users_ref.order_by('__lastFetched', direction=firestore.Query.DESCENDING).limit(limit)
    return getUsersFromRef(query)

In [22]:
def getSpecificUser(user):
    ref = db.collection('reddit').document('db').collection('users').document(user)
    doc = ref.get()
    doc_dict = doc.to_dict()
    return convertUserDict(doc_dict)